# Phenotype scores
Motivated by the Latini 2023 paper, I am going to annotate proteins in the network as activators or inhibitors of ‘apoptosis’ , 'differenciation' and ‘proliferation’ phenotypes using the recently published research `ProxPath` (Iannuccelli et al., 2022).

In [1]:
import ginsim
import biolqm
import pandas as pd
import numpy as np
from colomoto_jupyter import tabulate # for fixpoint table display
import matplotlib.pyplot as plt # for modifying plots
import seaborn as sns # for heatmap visualization
import xml.etree.ElementTree as ET # for parse the SBML file

This notebook has been executed using the docker image `colomoto/colomoto-docker:2024-01-01`

In [ ]:
model = "Palma2021_nophe.sbml"

In [2]:
def getnodes(model):
    # Load and parse the SBML file
    tree = ET.parse(model)
    root = tree.getroot()
    
    # Define the namespace for SBML Level 3 Version 1 Core and Qual
    ns = {
        'sbml': 'http://www.sbml.org/sbml/level3/version1/core',
        'qual': 'http://www.sbml.org/sbml/level3/version1/qual/version1'
    }
    
    # Find all qualitativeSpecies in the model
    qual_species_list = root.findall('.//qual:qualitativeSpecies', ns)
    
    # Extract the IDs of the qualitativeSpecies
    qual_species_ids = []
    for species in qual_species_list:
        species_id = species.attrib.get('{http://www.sbml.org/sbml/level3/version1/qual/version1}id')
        if species_id:
            qual_species_ids.append(species_id)
    return qual_species_ids

['AKT1', 'CDKN2A', 'BCL2', 'CCND1', 'CEBPA', 'DNMT3A', 'MAPK1', 'ETV6', 'FBXW7', 'FLT3', 'GSK3B', 'HOXA9', 'MEIS1', 'MYC', 'NPM1', 'SOX4', 'STAT5A', 'TP53']


>To compute the phenotypes’ activation status, we integrated with the OR logic (‘sum of scores’) the activation status of upstream nodes. As such, if two regulators of the same phenotype were linked in the same axis, we considered only the one at the end of the cascade.

In [3]:
def proxpath(qual_species_ids):
    # Load the ProxPath file
    file_path = 'Data/significant_paths_to_phenotypes.txt'
    df = pd.read_csv(file_path, sep='\t')
    
    # Define the target end nodes
    target_end_nodes = ['APOPTOSIS', 'DIFFERENTIATION', 'PROLIFERATION']
    
    # Function to find the closest gene to the EndNode
    def closest_gene(path_string, genes):
        # Split the path into components and reverse it (to start from the EndNode)
        components = path_string.split('--')[::-1]
        for component in components:
            # Check if the component contains any of the genes
            for gene in genes:
                if gene in component:
                    return gene
        return None
    
    # Filter rows first
    filtered_df = df[df['QueryNode'].isin(qual_species_ids) & df['EndNode'].isin(target_end_nodes)].copy()
    
    # Apply the function to each row in the filtered DataFrame
    filtered_df['Closest_Gene'] = filtered_df.apply(lambda row: closest_gene(row['Path_String'], qual_species_ids), axis=1)
    
    # Filter rows where QueryNode is the closest gene to the EndNode
    pheno = filtered_df[filtered_df['QueryNode'] == filtered_df['Closest_Gene']]
    
    # save the filtered data to a new file
    # pheno.to_csv('Phenotypes.txt', sep='\t', index=False)

    return pheno

,EndPathways,QueryNode,EndNode,Path_String,relations_path,Path_Score,Path_Length,Final_Effect,Effect,n,mean,sd,zscore,Closest_Gene
2894,APOPTOSIS,TP53,APOPTOSIS,TP53-->APOPTOSIS,SIGNOR-255678,0.300,1,1,up-regulates,75493,1.737625,0.526944,-2.728233,TP53
2896,APOPTOSIS,MAPK1,APOPTOSIS,MAPK1-->FOS-->JUN--|APOPTOSIS,SIGNOR-262996;SIGNOR-252087;SIGNOR-256560,0.578,3,-1,down-regulates,75493,1.737625,0.526944,-2.200663,MAPK1
2913,APOPTOSIS,BCL2,APOPTOSIS,BCL2--|APOPTOSIS,SIGNOR-249611,0.300,1,-1,down-regulates,75493,1.737625,0.526944,-2.728233,BCL2
2946,APOPTOSIS,MAPK1,APOPTOSIS,MAPK1--|FOXO-->APOPTOSIS,SIGNOR-252958;SIGNOR-252939,0.604,2,-1,down-regulates,75493,1.737625,0.526944,-2.151321,MAPK1
2987,APOPTOSIS,STAT5A,APOPTOSIS,STAT5A--|APOPTOSIS,SIGNOR-256583,0.300,1,-1,down-regulates,75493,1.737625,0.526944,-2.728233,STAT5A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54562,PROLIFERATION,MAPK1,PROLIFERATION,MAPK1--|FOXO--|PROLIFERATION,SIGNOR-252958;SIGNOR-252938,0.604,2,1,up-regulates,107507,1.681607,0.506006,-2.129632,MAPK1
54592,PROLIFERATION,TP53,PROLIFERATION,TP53-->CDKN1A--|CDK2--|CDKN1B--|PROLIFERATION,SIGNOR-173425;SIGNOR-149711;SIGNOR-154188;SIGN...,0.524,4,-1,down-regulates,107507,1.681607,0.506006,-2.287733,TP53
54640,PROLIFERATION,GSK3B,PROLIFERATION,GSK3B--|CTNNB1-->PROLIFERATION,SIGNOR-116528;SIGNOR-255654,0.454,2,-1,down-regulates,107507,1.681607,0.506006,-2.426071,GSK3B
54650,PROLIFERATION,AKT1,PROLIFERATION,AKT1--|RAC1-->PROLIFERATION,SIGNOR-252576;SIGNOR-255648,0.587,2,-1,down-regulates,107507,1.681607,0.506006,-2.163228,AKT1


>Then, we integrated the signal of the phenotype regulators proteins to compute the level of ‘apoptosis inhibition’ and ‘proliferation activation’ in each cell line. Integration, in this context, means computing the sum of the 'scores' of proteins that influence each phenotype, with an underlying assumption of equal importance for both inhibitors and activators (OR logic).

In [ ]:
def pheno_scores(simulation_results, pheno):
    
    # Remove duplicates: keep the first occurrence of each gene-phenotype pair
    pheno_unique = pheno.drop_duplicates(subset=['QueryNode', 'EndNode'])

    phenotypes = ['APOPTOSIS', 'DIFFERENTIATION', 'PROLIFERATION']    
    # Loop through each phenotype
    for phenotype in phenotypes:
        
        # Filter the 'pheno' dataframe for the current phenotype
        filtered_pheno = pheno_unique[pheno_unique['EndNode'] == phenotype]

        # Loop through each row in the filtered 'pheno' dataframe
        for idx, row in filtered_pheno.iterrows():
            gene = row['QueryNode']
            effect = row['Final_Effect']
            
            # Check if the gene is in the simulation results
            if gene in simulation_results.index:
                # Multiply the gene's simulation result by its effect (1 or -1)
                simulation_results.loc[phenotype] += simulation_results.loc[gene] * effect

    return simulation_results